In [1]:
import cv2
import os
import pydicom
os.chdir(r"D:\AIML PROJ") 
cwd = os.getcwd()
cwd
inputdir = 'train_dicom'
outdir = 'test_png'
out_dir_1='train_img'
#os.mkdir(outdir)

test_list = [ f for f in  os.listdir(inputdir)]
print(test_list)
#test_list=inputdir
for f in test_list:   # remove "[:10]" to convert all images 
    ds = pydicom.read_file(os.path.join(inputdir,f)) # read dicom image
    img = ds.pixel_array # get image array
    cv2.imwrite(os.path.join(outdir,f.replace('.dcm','.png')),img) # write png image

['f566c3b0-cfd0-4857-8155-6e4c44e5c442.dcm', 'f569df59-d9b3-4af9-8b26-657ee2abcd9a.dcm', 'f572b812-8956-414f-8c2d-ba18603f59de.dcm', 'f574b116-7f36-4ed5-9d6c-d885fd7b6caf.dcm', 'f574ff99-c50b-47e9-95fa-03d5eaae7069.dcm', 'f583fe64-e101-4a4d-a2d1-acaab5ae915f.dcm', 'f593cb8e-047a-48d1-b1f5-7d637fb94ce6.dcm', 'f603fefa-2256-40d5-890b-931a943ea8c2.dcm', 'f609f84e-7834-4854-925e-bb4b883e445a.dcm']


In [2]:
os.chdir(r"D:\AIML PROJ\test_png") 
cwd = os.getcwd()
cwd
img = cv2.imread("f566c3b0-cfd0-4857-8155-6e4c44e5c442.png", cv2.IMREAD_COLOR)
img.shape

(1024, 1024, 3)

In [3]:
import os 
import pandas as pd
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from pydicom import dcmread
import seaborn as sns

In [4]:
os.chdir(r"D:\AIML PROJ") 
cwd = os.getcwd() 
print("Current working directory:", cwd)

Current working directory: D:\AIML PROJ


In [5]:
datanew = np.load('image_array_final.npy',allow_pickle=True)

In [6]:
datanew

array({'7002c25e-83ce-4af1-9cdb-a9d4899d6104': array([[140, 151, 140, ...,  22,  21,  23],
       [145, 162, 149, ...,  20,  20,  20],
       [139, 137, 147, ...,  20,  20,  21],
       ...,
       [225, 214, 216, ..., 184, 185, 200],
       [231, 216, 216, ..., 184, 190, 201],
       [244, 234, 234, ..., 198, 203, 214]], dtype=uint8), '6a5d55af-fc99-4957-9404-f08e4150dc55': array([[  2,   3,   2, ...,   2,   3,   3],
       [  3,   3,   3, ...,   3,   3,   2],
       [  2,   3,   3, ...,   3,   3,   2],
       ...,
       [ 38,  33,  31, ..., 151, 149, 153],
       [ 38,  43,  38, ..., 156, 154, 153],
       [ 36,  33,  31, ..., 171, 163, 164]], dtype=uint8), '3f151141-a3bb-47aa-a213-4a90e8c55502': array([[3, 3, 3, ..., 2, 2, 2],
       [2, 3, 3, ..., 2, 2, 2],
       [3, 3, 3, ..., 2, 2, 2],
       ...,
       [2, 3, 3, ..., 5, 5, 3],
       [2, 3, 3, ..., 5, 5, 3],
       [3, 3, 3, ..., 5, 4, 4]], dtype=uint8), '4206173d-0db2-4f32-80fb-0bd99d5532e1': array([[ 25,  13,   8, ...,  81,

In [7]:
df=pd.read_csv('annotation.txt')

In [8]:
df.head(10)

,patientId,x,x-max,y,y-max,width,height,Target,class
0,7002c25e-83ce-4af1-9cdb-a9d4899d6104,115.25,168.75,51.75,194.50,53.50,142.75,1,Lung Opacity
1,6a5d55af-fc99-4957-9404-f08e4150dc55,51.00,80.00,112.00,127.75,29.00,15.75,1,Lung Opacity
2,3f151141-a3bb-47aa-a213-4a90e8c55502,135.00,191.25,87.25,196.50,56.25,109.25,1,Lung Opacity
3,4206173d-0db2-4f32-80fb-0bd99d5532e1,142.50,206.25,100.50,150.25,63.75,49.75,1,Lung Opacity
4,bcee03c8-2ff9-4b01-a6d6-a4ee99565d81,152.25,202.50,66.50,234.00,50.25,167.50,1,Lung Opacity
5,4bc8ce99-2420-48c2-a768-d067da67ced2,138.00,194.50,52.25,167.25,56.50,115.00,1,Lung Opacity
6,e5ee965b-7a79-4ceb-986c-a775a3c1d7a8,157.00,205.00,61.00,127.25,48.00,66.25,1,Lung Opacity
7,b12ab5b1-c824-4776-b859-df3ee1817269,168.75,221.75,130.50,188.75,53.00,58.25,1,Lung Opacity
8,cb815f3a-5014-4bbe-9e34-828149ba9f6e,142.00,198.00,64.75,169.00,56.00,104.25,1,Lung Opacity
9,e6a3f420-39d4-4042-8485-87024e7fb4f3,179.50,212.50,147.50,196.75,33.00,49.25,1,Lung Opacity


In [9]:
train_list=[]
patientId=[]
for pid in df['patientId']:
    patientId.append(pid)
    pid=pid+'.dcm'
    train_list.append(pid)

In [10]:
patientId

['7002c25e-83ce-4af1-9cdb-a9d4899d6104',
 '6a5d55af-fc99-4957-9404-f08e4150dc55',
 '3f151141-a3bb-47aa-a213-4a90e8c55502',
 '4206173d-0db2-4f32-80fb-0bd99d5532e1',
 'bcee03c8-2ff9-4b01-a6d6-a4ee99565d81',
 '4bc8ce99-2420-48c2-a768-d067da67ced2',
 'e5ee965b-7a79-4ceb-986c-a775a3c1d7a8',
 'b12ab5b1-c824-4776-b859-df3ee1817269',
 'cb815f3a-5014-4bbe-9e34-828149ba9f6e',
 'e6a3f420-39d4-4042-8485-87024e7fb4f3']

In [11]:
train_list

['7002c25e-83ce-4af1-9cdb-a9d4899d6104.dcm',
 '6a5d55af-fc99-4957-9404-f08e4150dc55.dcm',
 '3f151141-a3bb-47aa-a213-4a90e8c55502.dcm',
 '4206173d-0db2-4f32-80fb-0bd99d5532e1.dcm',
 'bcee03c8-2ff9-4b01-a6d6-a4ee99565d81.dcm',
 '4bc8ce99-2420-48c2-a768-d067da67ced2.dcm',
 'e5ee965b-7a79-4ceb-986c-a775a3c1d7a8.dcm',
 'b12ab5b1-c824-4776-b859-df3ee1817269.dcm',
 'cb815f3a-5014-4bbe-9e34-828149ba9f6e.dcm',
 'e6a3f420-39d4-4042-8485-87024e7fb4f3.dcm']

In [12]:
inputdir = 'stage_2_train_images'
for f in train_list:   # remove "[:10]" to convert all images 
    ds = pydicom.read_file(os.path.join(inputdir,f)) # read dicom image
    img = ds.pixel_array # get image array
    cv2.imwrite(os.path.join(out_dir_1,f.replace('.dcm','.png')),img) # write png image
    

In [14]:
images=[]
for f in patientId:
    f=f+'.png' 
    path_png=os.path.join(out_dir_1,f)
    images.append( path_png)

In [15]:
images

['train_img\\7002c25e-83ce-4af1-9cdb-a9d4899d6104.png',
 'train_img\\6a5d55af-fc99-4957-9404-f08e4150dc55.png',
 'train_img\\3f151141-a3bb-47aa-a213-4a90e8c55502.png',
 'train_img\\4206173d-0db2-4f32-80fb-0bd99d5532e1.png',
 'train_img\\bcee03c8-2ff9-4b01-a6d6-a4ee99565d81.png',
 'train_img\\4bc8ce99-2420-48c2-a768-d067da67ced2.png',
 'train_img\\e5ee965b-7a79-4ceb-986c-a775a3c1d7a8.png',
 'train_img\\b12ab5b1-c824-4776-b859-df3ee1817269.png',
 'train_img\\cb815f3a-5014-4bbe-9e34-828149ba9f6e.png',
 'train_img\\e6a3f420-39d4-4042-8485-87024e7fb4f3.png']

In [16]:
img = cv2.imread(images[0], cv2.IMREAD_COLOR)
cv2.imshow('image', img)
img.shape

(1024, 1024, 3)

In [17]:
X_df = pd.DataFrame({'patientId':patientId,'image_path':images})

In [18]:
X_df

,patientId,image_path
0,7002c25e-83ce-4af1-9cdb-a9d4899d6104,train_img\7002c25e-83ce-4af1-9cdb-a9d4899d6104...
1,6a5d55af-fc99-4957-9404-f08e4150dc55,train_img\6a5d55af-fc99-4957-9404-f08e4150dc55...
2,3f151141-a3bb-47aa-a213-4a90e8c55502,train_img\3f151141-a3bb-47aa-a213-4a90e8c55502...
3,4206173d-0db2-4f32-80fb-0bd99d5532e1,train_img\4206173d-0db2-4f32-80fb-0bd99d5532e1...
4,bcee03c8-2ff9-4b01-a6d6-a4ee99565d81,train_img\bcee03c8-2ff9-4b01-a6d6-a4ee99565d81...
5,4bc8ce99-2420-48c2-a768-d067da67ced2,train_img\4bc8ce99-2420-48c2-a768-d067da67ced2...
6,e5ee965b-7a79-4ceb-986c-a775a3c1d7a8,train_img\e5ee965b-7a79-4ceb-986c-a775a3c1d7a8...
7,b12ab5b1-c824-4776-b859-df3ee1817269,train_img\b12ab5b1-c824-4776-b859-df3ee1817269...
8,cb815f3a-5014-4bbe-9e34-828149ba9f6e,train_img\cb815f3a-5014-4bbe-9e34-828149ba9f6e...
9,e6a3f420-39d4-4042-8485-87024e7fb4f3,train_img\e6a3f420-39d4-4042-8485-87024e7fb4f3...


In [19]:
data_frame = df.merge(X_df, on='patientId', how='left')

In [20]:
data_frame.head(5)

,patientId,x,x-max,y,y-max,width,height,Target,class,image_path
0,7002c25e-83ce-4af1-9cdb-a9d4899d6104,115.25,168.75,51.75,194.50,53.50,142.75,1,Lung Opacity,train_img\7002c25e-83ce-4af1-9cdb-a9d4899d6104...
1,6a5d55af-fc99-4957-9404-f08e4150dc55,51.00,80.00,112.00,127.75,29.00,15.75,1,Lung Opacity,train_img\6a5d55af-fc99-4957-9404-f08e4150dc55...
2,3f151141-a3bb-47aa-a213-4a90e8c55502,135.00,191.25,87.25,196.50,56.25,109.25,1,Lung Opacity,train_img\3f151141-a3bb-47aa-a213-4a90e8c55502...
3,4206173d-0db2-4f32-80fb-0bd99d5532e1,142.50,206.25,100.50,150.25,63.75,49.75,1,Lung Opacity,train_img\4206173d-0db2-4f32-80fb-0bd99d5532e1...
4,bcee03c8-2ff9-4b01-a6d6-a4ee99565d81,152.25,202.50,66.50,234.00,50.25,167.50,1,Lung Opacity,train_img\bcee03c8-2ff9-4b01-a6d6-a4ee99565d81...
